## Задача 3 - Задача Генерализации ##

Новиков Евгений 120м

In [108]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

Данную задачу я решил сделать при помощи обработки текста - библиоткеки nltk, используя мешок слов. В дальнейшем, я поясню, какие у этого метода оказались недостатки.

Загрузка данных

In [109]:
raw_file = pd.read_csv('Назымское 35.csv', sep = ';', encoding='cp1251')

In [110]:
raw_file.dropna(inplace=True)

In [111]:
raw_file = raw_file.sort_values('ZK')

Токенизация

In [112]:
tokens_1 = []
for i in raw_file['LITO']: # разделить набор
    a = re.split(' |-',i)
    tokens_1.append(a)

Стемминг (можно было делать лемматизацию, но для задач несложной обработки небольшого количества слов (описаний образцов) этого должно хватить)

In [113]:
snowball = SnowballStemmer(language="russian")
token_2 = []
token_all = []
token_temp = []
for sample in tokens_1:
    for word in sample:
        temp = snowball.stem(word)
        if (temp == 'радиолярит'):
            temp = 'кремнист'
        elif (temp == 'известняк'):
            temp = 'карбонатн'
        token_all.append(temp)
        token_temp.append(temp)
    token_2.append(token_temp.copy())
    token_temp.clear()

**Векторизация - общая для всех образцов**

In [114]:
vectorizer = CountVectorizer().fit_transform(token_all)
vectors1 = vectorizer.toarray()
dict_ = {}
for word, vect in zip(token_all, vectors1):
    dict_[word] = vect

In [115]:
vectors1

array([[0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0,

In [116]:
dict_

{'пород': array([0, 0, 0, 0, 1], dtype=int64),
 'глинист': array([1, 0, 0, 0, 0], dtype=int64),
 'карбонатн': array([0, 1, 0, 0, 0], dtype=int64),
 'кремнист': array([0, 0, 0, 1, 0], dtype=int64),
 'керогенов': array([0, 0, 1, 0, 0], dtype=int64)}

**Векторизация - отдельно внутри групп**

In [117]:
for i in range(len(token_2)):
    for j in range(len(token_2[i])):
        if token_2[i][j] in dict_:
            token_2[i][j] = dict_[token_2[i][j]]

In [118]:
token_2 # видно разбиение по каждому образцу

[[array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 1, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 1, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 1, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 1, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 0, 1], dtype=int64),
  array([1, 0, 0, 0, 0], dtype=int64),
  array([0, 0, 0, 1, 0], dtype=int64)],
 [array([0, 0, 0, 

Сложение каждого слова в описании каждого образца, получается характеристика: образец - один вектор

In [119]:
token_new = []
for i in token_2:
    token_new.append(np.sum(i,0))
    
raw_file['LITO_token'] = token_new # теперь есть векторное представление каждого образца

Для удобства работы некотрую информацию можно перезаписать в списки

In [120]:
zk = list(raw_file['ZK'])
zp = list(raw_file['ZP'])
vec = list(raw_file['LITO_token'])
lit = list(raw_file['LITO'])

Функции

In [121]:
def norm(vec): # норма вектора
    x1,x2,x3,x4,x5 = vec
    return (x1**2 + x2**2 + x3**2 + x4**2 + x5**2)**0.5

def cos(): # расстояние между векторами - степень их похожести
    closeness = []
    for i in range(len(zk)-1):
        cos = np.dot(vec[i], vec[i+1])/((norm(vec[i])*norm(vec[i+1]) ) )
        #if (cos > 0.5):
        closeness.append(cos)
    closeness.append(0)
    return closeness

closeness = cos()
raw_file['Closeness'] = closeness # показывает, насколько текущий образец похож на следующий

In [122]:
raw_file

,ZK,ZP,LITO,LITO_token,Closeness
5,2635.00,2635.59,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
6,2635.59,2637.19,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
7,2637.19,2637.44,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
8,2637.44,2639.02,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",0.866025
9,2639.02,2641.40,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
10,2641.40,2642.17,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
11,2642.17,2643.04,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
12,2643.04,2643.91,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",0.577350
13,2643.91,2644.15,Радиоляриты,"[0, 0, 0, 1, 0]",0.577350
14,2644.15,2644.57,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",0.577350


Функция scale возвращает новые границы и новую литологию, исходя из необходимой степени разбиения образцов, необходимой кратности границ и сама создаёт новые названия для литотипов, совмещая названия литотипов в образцах, которые располагаются внутри этих границ

In [132]:
# discrete_factor в [метрах] - кратность границ
# closeness_barrier - насколько сильно необходимо соединять похожие пласты (=1 => объединять только с одинаковым описанием)

def scale(closeness_barrier = 1, discrete = 1, n = raw_file['ZP'].shape[0]): # функция для масштабирования
    # n - количество проходов по старым границам (равно числу старых границ)
    new_lito = [] # список новой литологии
    border = [] # border - список новых границ
    vec_sum = np.zeros(5)
    thick = 0
    for i in list(range(n)):
        thick = zp[i]
        if closeness[i] >= closeness_barrier:
            vec_sum += vec[i] # сложение векторов для генерации новых описаний литотипов
            thick += (zp[i] - zk[i])
        else:
            new_lito.append(vec_sum)
            vec_sum = np.zeros(5)
            vec_sum = vec[i]
            thick = thick - thick%discrete # кратность границ определенному числу
            border.append(thick)
    return border, new_lito

In [133]:
borders, lito_new_ = scale(0.8, 0.1) # заданы параметры близости > 0.8 и границы - до десятых (можно делать кратным любому числу)

Далее, чтобы нормализовать векторы новых названий литотипов в слова для декодирования нужно их привести нормированному виду (например, [1,0,0,1,1])

In [134]:
normalized = [] 
temp_ = []
for i in lito_new_: # здесь работа с коэффициентами
    sum_ = sum(i)
    for j in i:
        param = j/sum_ # приведение к единице
        if (param > 0.05):
            temp_.append(1) # эвристически - если содержание больше 0.05 - то оно решающее
        else:
            temp_.append(0)
    normalized.append(temp_.copy())
    temp_.clear()

Запись новых границ и новых литотипов в новую таблицу result

In [135]:
result = pd.DataFrame({'ZP': borders, 'LITO':lito_new_})
result['LITO_normalized'] = normalized
#result.drop('LITO', axis = 1)

Далее можно приступить к декодированию получившихся векторов - описаний образцов для получения слов

In [136]:
litology = []
temporary_ = []
for i in range(len(normalized)): # декодирование в соответствии со словарем dict_
    for j in range(len(normalized[i])):
        if (j == 0) and (normalized[i][j] == 1):
            temporary_.append('глинист')
        elif (j == 1) and (normalized[i][j] == 1):
            temporary_.append('карбонатн')
        elif (j == 2) and (normalized[i][j] == 1):
            temporary_.append('керогенов')
        elif (j == 3) and (normalized[i][j] == 1):
            temporary_.append('кремнист')
        #elif (j == 4) and (lito_str[i][j] == 1):  # слово 'пород' здесь не несет большого смысла
            #temporary_.append('пород')
    litology.append(temporary_.copy())
    temporary_.clear()
    
result['LITO_str'] = litology

Добавление кровли. Дополнительное устранение смещений в определении литотипов.

In [137]:
# добавление кровли для каждого пласта
ZK_ = []
ZK_.append(raw_file['ZK'].iloc[0])
for i in range(len(result['ZP'])-1):
    ZK_.append(result['ZP'].iloc[i])
    

result['ZK'] = ZK_
result = result[['ZK', 'ZP', 'LITO', 'LITO_normalized', 'LITO_str']] # упорядочивание
result[['ZK', 'ZP']] = result[['ZK', 'ZP']].shift(1) # сдвиг на один признак, поскольку оценка основывалась на сравнении со следующим, 
# следовательно в каждом образце записаны характеристики следующего образца (особенности работы функции cos())
result = result[:-1]
result.dropna(inplace = True)
result['ZK'].loc[0] = raw_file['ZK'].iloc[0] # отдельно нужно проставить первую границу
result = result.drop(['LITO', 'LITO_normalized'], axis = 1)

Итоговый результат
(можно поиграться с разными параметрами discrete_factor и closeness_barrier)

In [140]:
result # в данном случае при scale(0.8, 0.1)

,ZK,ZP,LITO_str
1,2635.0,2643.9,"[глинист, кремнист]"
2,2643.9,2644.1,[кремнист]
3,2644.1,2644.5,"[глинист, кремнист]"
4,2644.5,2644.8,"[глинист, керогенов, кремнист]"
5,2644.8,2646.8,"[глинист, керогенов, кремнист]"
6,2646.8,2647.0,"[глинист, карбонатн, керогенов, кремнист]"
7,2647.0,2656.5,"[глинист, кремнист]"


In [139]:
raw_file # для сравнения с исходной таблицей

,ZK,ZP,LITO,LITO_token,Closeness
5,2635.00,2635.59,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
6,2635.59,2637.19,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
7,2637.19,2637.44,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",1.000000
8,2637.44,2639.02,Породы глинисто-карбонатно-кремнистые,"[1, 1, 0, 1, 1]",0.866025
9,2639.02,2641.40,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
10,2641.40,2642.17,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
11,2642.17,2643.04,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",1.000000
12,2643.04,2643.91,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",0.577350
13,2643.91,2644.15,Радиоляриты,"[0, 0, 0, 1, 0]",0.577350
14,2644.15,2644.57,Породы глинисто-кремнистые,"[1, 0, 0, 1, 1]",0.577350


Вывод получается достаточно неплохо. Единственный неостаток - не учитывается очередность пород в описании, поскольку у стееминга нет нормального джекодирования, он не сохраняте порядок слов при кодировке (векторизации), поэтому часть информации теряется.